<a href="https://colab.research.google.com/github/Ayo-Cyber/Bot_00/blob/main/Hate_speech_Detection_algorothm_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing Of Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv("/content/labeled_data.csv")

In [ ]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,24783.0,12681.192027,7299.553863,0.0,6372.5,12703.0,18995.5,25296.0
count,24783.0,3.243473,0.883060,3.0,3.0,3.0,3.0,9.0
hate_speech,24783.0,0.280515,0.631851,0.0,0.0,0.0,0.0,7.0
offensive_language,24783.0,2.413711,1.399459,0.0,2.0,3.0,3.0,9.0
neither,24783.0,0.549247,1.113299,0.0,0.0,0.0,0.0,9.0
class,24783.0,1.110277,0.462089,0.0,1.0,1.0,1.0,2.0


In [ ]:
df_transformed = df[['class','tweet']]
y = df_transformed.iloc[: , :-1].values

##Encoding And Preprocessing of Variables 

In [ ]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder() , [0])] , remainder='passthrough')
y = np.array(ct.fit_transform(y))

In [ ]:
print(y)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
y_df = pd.DataFrame(y)
y_hate = np.array(y_df[0])
y_offensive = np.array(y_df[1])

In [ ]:
print(y_hate)
print(y_offensive)

[0. 0. 0. ... 0. 0. 0.]
[0. 1. 1. ... 1. 1. 0.]


##Cleaning Data



In [ ]:
corpus = []
for i in range(0,24783):
  review = re.sub('[^a-zA-Z]',' ',df_transformed['tweet'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
cv = CountVectorizer(max_features=2000)
x = cv.fit_transform(corpus).toarray()

##Splitting Of Data Into Train And Test

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y_hate ,test_size=0.3 , random_state = 0)

##Model Building

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
classifier_np = GaussianNB()
classifier_np.fit(x_train , y_train)

GaussianNB()

In [ ]:
classifier_lr = LogisticRegression()
classifier_lr.fit(x_train,y_train)

LogisticRegression()

In [ ]:
classifier_svm = SVC()
classifier_svm.fit(x_train,y_train)

KeyboardInterrupt: ignored

##Building A Confusion Matrix To test Models 

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred_np = classifier_np.predict(x_test)
cm = confusion_matrix(y_pred_np , y_test)
print(cm)

[[3289  168]
 [3719  259]]


In [ ]:
y_pred_lr = classifier_lr.predict(x_test)
cm = confusion_matrix(y_pred_lr , y_test)
print(cm)

[[6911  347]
 [  97   80]]


In [ ]:
y_pred_svm= classifier_svm.predict(x_test)
cm = confusion_matrix(y_pred_svm , y_test)
print(cm)

[[6964  372]
 [  44   55]]
